## Import Google LLM GenAI Gemini API Key

In [1]:
import os
os.environ["GOOGLE_API_KEY"] = "Insert GenAI API Key here"

#enable additional feature, LangSmith, which allows tracking of what's running behind
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = 'Insert LangSmith API here'

## Import Google GenAi Embedding Feature

In [2]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
#embeddings.embed_query("hello, world!")

## Load CSV File with LangChain

In [3]:
from langchain.document_loaders import CSVLoader
loader = CSVLoader(file_path='./decoded_sacom.csv', encoding="utf-8")
data = loader.load()

In [4]:
print(type(data))
print(len(data))
data[0]

<class 'list'>
14385


Document(page_content='Org ID: 193932\nOrg Name: RSL Ardrossan Sub Branch\nAKA: Ardrossan RSL----Returned & Services League Ardrossan\nAcronym: \nFormer Name: \nS Street Addr 1: RSL Hall, West Tce\nS Street Addr 2: \nS Suburb: Ardrossan\nS State: South Australia\nS Postcode: 5571\nPhone: 08 8837 3596++\nMobile: \nEmail: ardrossan@rslsa.org.au\nWebsite: http://rslsa.org.au/stores/ardrossan\nParent Body: \nParent Body URL: \nOpen Hours: \nWheelchair Access: \nToilets Access: \nDisabled Parking: \nServices: Welfare and pensions support for ex-servicemen and their families\r\\\nSocial and recreational activities\r\\\nCommemoration activities - ANZAC Day, Remembrance Day and other significant events\nABN: 25166174779\nLocal Community dir: Service Clubs\nAdelaide Hills dir: \nOnkaparinga dir: \nSubjects: Ex-Defence Service Groups----Halls For Hire----Social & Activity Groups----Support & Resource Groups----Veterans\nPrimary Category: Recreation\nCouncil: Yorke Peninsula Council', metadata={'

## Use FAISS for vector store

In [5]:
from langchain.vectorstores import FAISS

vectorstore = FAISS.from_documents(data, embeddings)

## Check how many vector store

In [6]:
print(vectorstore.index.ntotal)

14385


## Simple Search using FAISS to fetch relevant information

In [7]:
#Enter Search Query
query = "Hall for Hire, Prospect"

#Search the vector store with query, k is the number of results we want FAISS to return
docs = vectorstore.similarity_search_with_score(query, k=5)

#Print the first relevant document, which has the highest similarity score
print(docs[0][0].page_content) #The row's content
print("Similarity score: "+str(docs[0][1])) #The similarity score

Org ID: 197897
Org Name: Prospect Town Hall
AKA: Eliza Hall
Acronym: 
Former Name: 
S Street Addr 1: Payinthi Civic Centre,
S Street Addr 2: 128 Prospect Rd
S Suburb: Prospect
S State: South Australia
S Postcode: 5082
Phone: 08 8269 5355++
Mobile: 
Email: admin@prospect.sa.gov.au
Website: https://www.spacetoco.com/space/eliza-hall
Parent Body: 
Parent Body URL: 
Open Hours: Mon - Sun 7am - 12am
Wheelchair Access: 1.0
Toilets Access: 1.0
Disabled Parking: 1.0
Services: Venue and meeting rooms for hire
ABN: 
Local Community dir: Halls & Venues for Hire
Adelaide Hills dir: 
Onkaparinga dir: 
Subjects: Halls For Hire----Meeting Rooms
Primary Category: Community Organisation & Development
Council: City of Prospect
Similarity score: 0.5422785


In [8]:
print (type(docs), len(docs))
print (type(docs[0][0]))
print (type(docs[0][0].page_content))

<class 'list'> 5
<class 'langchain_core.documents.base.Document'>
<class 'str'>


## RETRIEVAL

In [9]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 10})

In [10]:
retrieved_docs = retriever.invoke("Food bank in Prospect?")

In [11]:
len(retrieved_docs)

10

In [12]:
print(retrieved_docs[0].page_content)

Org ID: 197495
Org Name: St Vincent de Paul Society Family Centre - Prospect
AKA: Society of St Vincent de Paul Family Centre - Prospect
Acronym: 
Former Name: 
S Street Addr 1: 92a Prospect Rd
S Street Addr 2: 
S Suburb: Prospect
S State: South Australia
S Postcode: 5082
Phone: 08 8269 5817++
Mobile: 
Email: svdp@svdpsa.org.au
Website: www.vinnies.org.au
Parent Body: 
Parent Body URL: 
\pen Hours: Mon - Sat 9am - 4pm, Closed public holidays
Wheelchair Access: 
Toilets Access: 
Disabled Parking: 
\ervices: Distribution of donated goods to people in need - clothing, crockery, blankets, pillows, sheets and towels
Good quality second hand goods available for sale to low income families
ABN: 
Local Community dir: Food & Clothing Assistance
Adelaide Hills dir: 
Onkaparinga dir: 
Subjects: Clothing Assistance----Furniture Assistance----Goods & Services----Household Goods Assistance----Opportunity Shops
Primary Category: Material & Practical Needs
Council: City of Prospect


In [13]:
for i in range (len(retrieved_docs)):
    print (retrieved_docs[i].page_content)

Org ID: 197495
Org Name: St Vincent de Paul Society Family Centre - Prospect
AKA: Society of St Vincent de Paul Family Centre - Prospect
Acronym: 
Former Name: 
S Street Addr 1: 92a Prospect Rd
S Street Addr 2: 
S Suburb: Prospect
S State: South Australia
S Postcode: 5082
Phone: 08 8269 5817++
Mobile: 
Email: svdp@svdpsa.org.au
Website: www.vinnies.org.au
Parent Body: 
Parent Body URL: 
\pen Hours: Mon - Sat 9am - 4pm, Closed public holidays
Wheelchair Access: 
Toilets Access: 
Disabled Parking: 
\ervices: Distribution of donated goods to people in need - clothing, crockery, blankets, pillows, sheets and towels
Good quality second hand goods available for sale to low income families
ABN: 
Local Community dir: Food & Clothing Assistance
Adelaide Hills dir: 
Onkaparinga dir: 
Subjects: Clothing Assistance----Furniture Assistance----Goods & Services----Household Goods Assistance----Opportunity Shops
Primary Category: Material & Practical Needs
Council: City of Prospect
Org ID: 197391
Org 

## Generation

In [45]:
#import LLM
from langchain_google_genai import ChatGoogleGenerativeAI,HarmBlockThreshold,HarmCategory

llm = ChatGoogleGenerativeAI(temperature=0, model="gemini-pro",
    safety_settings={
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
        })

In [46]:
llm

ChatGoogleGenerativeAI(model='gemini-pro', temperature=0.0, safety_settings={<HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: 9>: <HarmBlockThreshold.BLOCK_NONE: 4>, <HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: 10>: <HarmBlockThreshold.BLOCK_NONE: 4>, <HarmCategory.HARM_CATEGORY_HATE_SPEECH: 8>: <HarmBlockThreshold.BLOCK_NONE: 4>, <HarmCategory.HARM_CATEGORY_HARASSMENT: 7>: <HarmBlockThreshold.BLOCK_NONE: 4>}, client=genai.GenerativeModel(
    model_name='models/gemini-pro',
    generation_config={},
    safety_settings={},
    tools=None,
))

## Use LangChain default RAG prompt

In [58]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [59]:
example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()
example_messages

[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: filler question \nContext: filler context \nAnswer:")]

In [60]:
print(example_messages[0].content)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: filler question 
Context: filler context 
Answer:


In [61]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [62]:
for chunk in rag_chain.stream("I need support for drug addiction"):
    print(chunk, end="", flush=True)

Narcotics Anonymous provides support for people with drug problems. Their phone number is 1300 652 820. Family Drug Support also offers support groups, a support line, and an interactive website on coping. Their phone number is 02 4782 9222.

## Use a prompt template with clear instruction on how the Chatbot should reply

In [63]:
from langchain.prompts import ChatPromptTemplate
template = """You are an customer service assistant for question-answering tasks.
Answer the question based only on the following context delimited by triple backtick.
context:```{context}```
If you cannot find the answer, tell them to be more precise.
You should reply in a friendly way.
Your should also provide the website link in the following format:
Org Name https://sacommunity.org/org/Org ID
Question: {question}
Answer:
"""

In [64]:
prompt = ChatPromptTemplate.from_template(template)
prompt

ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='You are an customer service assistant for question-answering tasks.\nAnswer the question based only on the following context delimited by triple backtick.\ncontext:```{context}```\nIf you cannot find the answer, tell them to be more precise.\nYou should reply in a friendly way.\nYour should also be providing the website link for them in the following format:\nOrg Name https://sacommunity.org/org/Org ID\nQuestion: {question}\nAnswer:\n'))])

In [65]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [85]:
for chunk in rag_chain.stream("I want to quit smoking"):
    print(chunk, end="", flush=True)

Org Name Quitline https://sacommunity.org/org/202205